In [1]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from textblob import TextBlob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

c:\Users\markham\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
test_data = pd.read_csv('data/processed_articles_2021_V1.csv')
test_data['content'] = test_data['title'] + test_data['body']
test_data = test_data.drop(columns=['title','body'])
test_data['date'] = pd.to_datetime(test_data['date'])
test_data = test_data.sort_values(by='date')
test_data['content'] = test_data['content'].apply(lambda x: ' '.join(x.split(',')))
stop_words = set(stopwords.words('english'))
stop_words_list = [ 'msci','iran','states','italy','united','china','usa','us','america','american','americans','chinese','china','russia','russian','putin','vladimir','trump','donald','biden','joe','ukraine','ukrainian','ukrainians','ukraines','ukraine','say','jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec','mon','tue','wed','thu','fri','sat','sun','monday','tuesday','wednesday','thursday','friday','saturday','sunday','january','february','march','april','may','june','july','august','september','october','november','december','today','yesterday','tomorrow','week','month','year','time','day','weekend','morning','afternoon','evening','night','news','new','news']
list_numbers = ['liga','len','nhl','one','two','three','four','five','six','seven','eight','nine','ten','hundred','thousand','million','billion','trillion']
country_list = ['China', 'United States of America', 'United Kingdom', 'France', 'Germany', 'Japan', 'Russia', 'Australia', 'Canada', 'India', 'Brazil', 'Italy', 'Spain', 'South Korea', 'Mexico', 'Netherlands', 'Switzerland', 'Sweden', 'Norway', 'Denmark', 'Finland', 'Greece', 'Ireland', 'Portugal', 'Poland', 'Ukraine', 'Romania', 'Belgium', 'Austria', 'Turkey', 'Saudi Arabia', 'United Arab Emirates', 'Iran', 'Iraq', 'Israel', 'Egypt', 'South Africa', 'Argentina', 'Venezuela', 'Thailand', 'Malaysia', 'Singapore', 'Indonesia', 'Philippines', 'Pakistan', 'Bangladesh', 'Nigeria', 'Kenya', 'Tanzania', 'Uganda']
stop_words.update(stop_words_list)
stop_words.update(list_numbers)
stop_words.update(country_list)
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in (stop_words)]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
test_data['content'] = test_data['content'].apply(lambda x: ' '.join([word for word in x.split() if word != 'nan']))
test_data['content_tokens'] = test_data['content'].apply(lambda x: x.split())

In [3]:
# 参数搜索范围
vector_sizes = [50, 60,70, 80,90,100]  # 词向量维度
windows = [5,6,7,8]  # 窗口大小
min_counts = [2, 3, 4,5]  # 最小出现次数
num_clusters_list = [2,3] # K-Means 聚类个数
N_values = [ 10, 15, 20,25]  # 选择最重要的 N 个词

best_score = -1
best_params = {}

# 遍历所有参数组合
for vector_size in vector_sizes:
    for window in windows:
        for min_count in min_counts:
            # 训练 Word2Vec 模型
            model = Word2Vec(sentences=test_data['content_tokens'], 
                             vector_size=vector_size, window=window, 
                             min_count=min_count, workers=8)
            
            # 初始化 VADER 分析器
            sid = SentimentIntensityAnalyzer()

            # 获取积极和消极词
            positive_seed_words = [word for word, score in sid.lexicon.items() if score > 1.5 and word.isalpha()]
            negative_seed_words = [word for word, score in sid.lexicon.items() if score < -1.5 and word.isalpha()]

            # 过滤不在 Word2Vec 词汇表中的种子词
            positive_seed_words = [word for word in positive_seed_words if word in model.wv]
            negative_seed_words = [word for word in negative_seed_words if word in model.wv]

            # 计算种子词相似度
            word_similarities = {}
            for word in model.wv.index_to_key:
                pos_sim = max([model.wv.similarity(word, seed) for seed in positive_seed_words], default=0)
                neg_sim = max([model.wv.similarity(word, seed) for seed in negative_seed_words], default=0)
                word_similarities[word] = max(pos_sim, neg_sim)

            for N in N_values:
                # 选择最重要的 N 个词
                test_data['important_words'] = test_data['content_tokens'].apply(
                    lambda tokens: sorted(
                        [token for token in tokens if token in word_similarities],
                        key=lambda x: word_similarities[x],
                        reverse=True
                    )[:N]
                )

                # 将文本转换为向量
                def text_to_vector(text, model):
                    vectors = [model.wv[word] for word in text if word in model.wv]
                    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)

                test_data['vector'] = test_data['important_words'].apply(lambda x: text_to_vector(x, model))

                # 进行 K-Means 聚类并计算 Silhouette Score
                X = np.array(test_data['vector'].tolist())


                for num_clusters in num_clusters_list:
                    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
                    labels = kmeans.fit_predict(X)

                    score = silhouette_score(X, labels)  # 计算轮廓系数
                    print(f"vector_size={vector_size}, window={window}, min_count={min_count}, N={N}, num_clusters={num_clusters} -> silhouette_score={score:.4f}")

                    # 更新最佳参数
                    if score > best_score:
                        best_score = score
                        best_params = {
                            "vector_size": vector_size,
                            "window": window,
                            "min_count": min_count,
                            "N": N,
                            "num_clusters": num_clusters,
                            "silhouette_score": score
                        }

# 输出最佳参数
print("\n✅ Best Parameters:")
print(best_params)

vector_size=50, window=5, min_count=2, N=10, num_clusters=2 -> silhouette_score=0.4211
vector_size=50, window=5, min_count=2, N=10, num_clusters=3 -> silhouette_score=0.3826
vector_size=50, window=5, min_count=2, N=15, num_clusters=2 -> silhouette_score=0.4108
vector_size=50, window=5, min_count=2, N=15, num_clusters=3 -> silhouette_score=0.3781
vector_size=50, window=5, min_count=2, N=20, num_clusters=2 -> silhouette_score=0.4046
vector_size=50, window=5, min_count=2, N=20, num_clusters=3 -> silhouette_score=0.3818
vector_size=50, window=5, min_count=2, N=25, num_clusters=2 -> silhouette_score=0.3922
vector_size=50, window=5, min_count=2, N=25, num_clusters=3 -> silhouette_score=0.3731
vector_size=50, window=5, min_count=3, N=10, num_clusters=2 -> silhouette_score=0.4039
vector_size=50, window=5, min_count=3, N=10, num_clusters=3 -> silhouette_score=0.3640
vector_size=50, window=5, min_count=3, N=15, num_clusters=2 -> silhouette_score=0.3971
vector_size=50, window=5, min_count=3, N=15